# Projeto Marinha do Brasil

## Laboratório de Processamento de Sinais - UFRJ

### Autor: Vinícius dos Santos Mello <viniciusdsmello@poli.ufrj.br>

In [1]:
import os
import sys
import time
from datetime import datetime, timedelta
sys.path.insert(0,'..')

from noveltyDetectionConfig import CONFIG
import numpy as np
import itertools
import multiprocessing
import pprint 

from Functions.email_utils import EmailConnection, Email
from SAENoveltyDetectionAnalysis import SAENoveltyDetectionAnalysis

num_processes = multiprocessing.cpu_count()

from Functions.telegrambot import Bot

my_bot = Bot("lisa_thebot")

# Enviroment variables
data_path = CONFIG['OUTPUTDATAPATH']
results_path = CONFIG['PACKAGE_NAME']

training_params = {
    "Technique": "StackedAutoEncoder",
    "TechniqueParameters": {
        "allow_change_weights": True #False
    },
    "DevelopmentMode": False,
    "DevelopmentEvents": 400,
    "NoveltyDetection": True,
    "InputDataConfig": {
        "database": "4classes",
        "n_pts_fft": 1024,
        "decimation_rate": 3,
        "spectrum_bins_left": 400,
        "n_windows": 1,
        "balance_data": True
    },
    "OptmizerAlgorithm": {
        "name": "Adam",
        "parameters": {
            "learning_rate": 0.001,
            "beta_1": 0.90,
            "beta_2": 0.999,
            "epsilon": 1e-08,
            "learning_decay": 1e-6,
            "momentum": 0.3,
            "nesterov": True
        }
    },
    "HyperParameters": {
        "n_folds": 4,#10, #4,
        "n_epochs": 500,#500, #300,
        "n_inits": 1,#1, #2,
        "batch_size": 128,#128, #256,
        "kernel_initializer": "uniform",
        "encoder_activation_function": "tanh", #"relu",
        "decoder_activation_function": "linear",
        "classifier_output_activation_function": "softmax",
        "norm": "mapstd",
        "metrics": ["accuracy"],
        "loss": "mean_squared_error",
        "dropout": False,
        "dropout_parameter": 0.00,
        "regularization": None,
        "regularization_parameter": 0.00
    },
    "callbacks": {
        "EarlyStopping": {
            "patience": 30,
            "monitor": "val_loss"
        }
    }
}
analysis = SAENoveltyDetectionAnalysis(parameters=training_params,
                                       model_hash="",
                                       load_hash=False, load_data=True, verbose=True)
all_data, all_trgt, trgt_sparse = analysis.getData()

SAE = analysis.createSAEModels()

trn_data = analysis.trn_data
trn_trgt = analysis.trn_trgt
trn_trgt_sparse = analysis.trn_trgt_sparse

Using TensorFlow backend.


Creating /home/vinicius.mello/Workspace/SonarAnalysis_2/Results/NoveltyDetection/StackedAutoEncoder/outputs/7bec0361cae6311c72f300bfe418d27f980780faaee26f3b6ee4ea9ceccaa265
Creating /home/vinicius.mello/Workspace/SonarAnalysis_2/Results/NoveltyDetection/StackedAutoEncoder/outputs/7bec0361cae6311c72f300bfe418d27f980780faaee26f3b6ee4ea9ceccaa265/AnalysisFiles
Creating /home/vinicius.mello/Workspace/SonarAnalysis_2/Results/NoveltyDetection/StackedAutoEncoder/outputs/7bec0361cae6311c72f300bfe418d27f980780faaee26f3b6ee4ea9ceccaa265/Pictures
Saving /home/vinicius.mello/Workspace/SonarAnalysis_2/Results/NoveltyDetection/StackedAutoEncoder/outputs/7bec0361cae6311c72f300bfe418d27f980780faaee26f3b6ee4ea9ceccaa265/parameters.json
[+] Time to read data file: 1.9878411293 seconds
Qtd event of A is 12939
Qtd event of B is 29352
Qtd event of C is 11510
Qtd event of D is 23760

Biggest class is B with 29352 events
Total of events in the dataset is 77561
Balacing data...
DataHandler Class: CreateEvents

In [2]:
pp = pprint.PrettyPrinter(indent=1)
print analysis.model_hash
print analysis.getBaseResultsPath()
pp.pprint(analysis.parameters)

7bec0361cae6311c72f300bfe418d27f980780faaee26f3b6ee4ea9ceccaa265
/home/vinicius.mello/Workspace/SonarAnalysis_2/Results/NoveltyDetection/StackedAutoEncoder/outputs/7bec0361cae6311c72f300bfe418d27f980780faaee26f3b6ee4ea9ceccaa265
{'DevelopmentEvents': 400,
 'DevelopmentMode': False,
 'HyperParameters': {'batch_size': 128,
                     'classifier_output_activation_function': 'softmax',
                     'decoder_activation_function': 'linear',
                     'dropout': False,
                     'dropout_parameter': 0.0,
                     'encoder_activation_function': 'tanh',
                     'kernel_initializer': 'uniform',
                     'loss': 'mean_squared_error',
                     'metrics': ['accuracy'],
                     'n_epochs': 500,
                     'n_folds': 4,
                     'n_inits': 1,
                     'norm': 'mapstd',
                     'regularization': None,
                     'regularization_parameter': 0.0}

## Testar topologia
1_inits_mapstd_norm_500_epochs_128_batch_size_tanh_hidden_activation_linear_output_activation_accuracy_metric_mean_squared_error_loss

Trainable = True

(n_inits=1,
hidden_activation='tanh', # others tanh, relu, sigmoid, linear 
output_activation='linear',
n_epochs=500, #500
patience=30, # 30
batch_size=128, #128
verbose=False)

#### Perform the training of the model

In [ ]:
trn_data = analysis.trn_data
trn_trgt = analysis.trn_trgt
trn_trgt_sparse = analysis.trn_trgt_sparse

inovelty = 0

data=trn_data[inovelty]
trgt=trn_trgt[inovelty]

ifold, classifier, trn_desc = SAE[inovelty].train_classifier(data  = data,
                                                             trgt  = trgt,
                                                             ifold = 0,
                                                             hidden_neurons = [200,1],
                                                             layer = 1)

In [ ]:
for inovelty in range(len(analysis.class_labels)):
    startTime = time.time()
    
    data=trn_data[inovelty]
    trgt=trn_trgt[inovelty]

    analysis.train(layer=1,
                   inovelty=inovelty,
                   fineTuning=True,
                   trainingType="neuronSweep", #foldSweep, neuronSweep, normal
                   hidden_neurons=[400],
                   neurons_variation_step=100,
                   numThreads=10,
                   model_hash=analysis.model_hash)
    
    duration = str(timedelta(seconds=float(time.time() - startTime)))
    message = "Technique: Stacked Autoencoder\n"
    message = message + "Training Type: Neurons Sweep\n"
    message = message + "Novelty Class: {}\n".format(analysis.class_labels[inovelty])
    message = message + "Duration: {}\n".format(duration)
    try:
        my_bot.sendMessage(message)
    except Exception as e:
        print("Erro ao enviar mensagem. Erro: " + str(e))
    print "The training of the model for novelty class {0} took {1} to be performed\n".format(analysis.class_labels[inovelty], duration)

python sae_train.py --layer 1 --novelty 0 --finetunning 1 --threads 10 --type neuronSweep --hiddenNeurons 400 --neuronsVariationStep 100 --modelhash 7bec0361cae6311c72f300bfe418d27f980780faaee26f3b6ee4ea9ceccaa265


# Pre-training analysis

### Mean Squared Error analysis for Pre-training Step with a neuron variation at autoencoder

In [ ]:
# Neuron variation x MSE Divergence
%matplotlib inline 

from sklearn import metrics
from sklearn import preprocessing
from sklearn.externals import joblib
from Functions.StatisticalAnalysis import KLDiv, EstPDF
import matplotlib.pyplot as plt

# Choose layer 
layer = 1

# Choose neurons topology
# hidden_neurons = range(400,0,-50) + [2]
hidden_neurons = [400,200]

step = 100
neurons_mat = [1] + range(step,hidden_neurons[layer-1]+step,step)
neurons_mat = neurons_mat[:len(neurons_mat)-layer+2]

analysis_name = 'mean_squared_error_%i_layer'%(layer)
analysis_file = os.path.join(analysis.getBaseResultsPath(), "AnalysisFiles", analysis_name + ".jbl")    

# os.remove(analysis_file)

verbose = True

# Plot parameters
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['legend.numpoints'] = 1
plt.rcParams['legend.handlelength'] = 3
plt.rcParams['legend.borderpad'] = 0.3
plt.rcParams['legend.fontsize'] = 14
m_colors = ['b', 'r', 'g', 'y']
figsize = (10,5)

mse = {}
mse_known = np.zeros([len(analysis.class_labels), analysis.n_folds, len(neurons_mat)])
mse_novelty = np.zeros([len(analysis.class_labels), analysis.n_folds, len(neurons_mat)])

if not os.path.exists(analysis_file):
    for inovelty in range(len(analysis.class_labels)):
        n_bins = 100
        for ineuron in neurons_mat: 
            if ineuron == 0:
                ineuron = 1
            def getMSE(ifold):
                train_id, test_id = analysis.CVO[inovelty][ifold]

                # normalize known classes
                if analysis.parameters["HyperParameters"]["norm"] == "mapstd":
                    scaler = preprocessing.StandardScaler().fit(all_data[all_trgt!=inovelty][train_id,:])
                elif analysis.parameters["HyperParameters"]["norm"] == "mapstd_rob":
                    scaler = preprocessing.RobustScaler().fit(all_data[all_trgt!=inovelty][train_id,:])
                elif analysis.parameters["HyperParameters"]["norm"] == "mapminmax":
                    scaler = preprocessing.MinMaxScaler().fit(all_data[all_trgt!=inovelty][train_id,:])

                known_data = scaler.transform(all_data[all_trgt!=inovelty][test_id,:])
                novelty_data = scaler.transform(all_data[all_trgt==inovelty])

                model = SAE[inovelty].get_model(data=all_data, trgt=all_trgt,
                                                hidden_neurons=hidden_neurons[:layer-1]+[ineuron],
                                                layer=layer, ifold=ifold)

                known_output = model.predict(known_data)
                novelty_output = model.predict(novelty_data)

                mseKnown = metrics.mean_squared_error(known_data, known_output)
                mseNovelty = metrics.mean_squared_error(novelty_data, novelty_output)

                return ifold, mseKnown, mseNovelty

            # Start Parallel processing
            p = multiprocessing.Pool(processes=num_processes)

            folds = range(len(analysis.CVO[inovelty]))
            if verbose:
                print '[*] Calculating Mean Squared Error ...'
            mse[ineuron] = p.map(getMSE, folds)

            for ifold in range(analysis.n_folds):
                mse_known[inovelty,:, neurons_mat.index(ineuron)] = mse[ineuron][ifold][1]
                mse_novelty[inovelty,:, neurons_mat.index(ineuron)] = mse[ineuron][ifold][2]

            p.close()
            p.join()
    
    joblib.dump([neurons_mat,mse_known,mse_novelty],analysis_file,compress=9)
else:
    [neurons_mat, mse_known, mse_novelty] = joblib.load(analysis_file)

for inovelty in range(len(analysis.class_labels)):
    
    # Plot results    
    fig = plt.subplots(figsize=figsize)
    ax = plt.subplot(1,1,1)

    ax.errorbar(neurons_mat, np.mean(mse_known[inovelty], axis=0),
                np.std(mse_known[inovelty], axis=0),fmt='o-',
                color='b',alpha=0.7,linewidth=2.5,
                label='MSE Known Test Data')
    ax.errorbar(neurons_mat, np.mean(mse_novelty[inovelty], axis=0),
                np.std(mse_novelty[inovelty], axis=0),fmt='.--',
                color='r',alpha=0.7,linewidth=2.5,
                label='MSE Novelty Data')

    ax.set_title('MSE x Neurons (Class {} as novelty) - Layer {}'.format(analysis.class_labels[inovelty], layer),
                 fontsize=14, fontweight='bold')

    ax.set_ylabel('Mean Squared Error', fontsize=20)
    ax.set_xlabel('Neurons', fontsize=20)
    ax.grid()
    ax.legend()
    plt.tight_layout()

    plt.show()

#Save the figure
# file_name = pict_results_path+'/'+current_analysis+'_%i_novelty_%s_neurons_'%(inovelty,neurons_str)+trn_params.get_params_str()+'.pdf'
# plt.savefig(file_name)

### Mutual Information analysis for Pre-training Step with a neuron variation at autoencoder

### Scatter Plot of Original data vs Reconstructed data for the features with the highest mean squared error

### Reconstruction of the Lofargram of a novelty class

# Fine Tuning Analysis

### SP Index analysis for a Fine Tuning Step with a neuron variation at autoencoder

In [ ]:
%matplotlib inline 

from sklearn import metrics
from sklearn import preprocessing
from sklearn.externals import joblib
from Functions.StatisticalAnalysis import KLDiv, EstPDF
import matplotlib.pyplot as plt

# Choose layer 
layer = 1

# Choose neurons topology
hidden_neurons = [400]

step = 100
neurons_mat = [1] + range(step,hidden_neurons[layer-1]+step,step)
neurons_mat = neurons_mat[:len(neurons_mat)-layer+2]

analysis_name = 'sp_index_%i_layer'%(layer)
analysis_file = os.path.join(analysis.getBaseResultsPath(), "AnalysisFiles", analysis_name + ".jbl")    

if os.path.exists(analysis_file):
    os.remove(analysis_file)
    
verbose = True

# Plot parameters
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['legend.numpoints'] = 1
plt.rcParams['legend.handlelength'] = 3
plt.rcParams['legend.borderpad'] = 0.3
plt.rcParams['legend.fontsize'] = 14
m_colors = ['b', 'r', 'g', 'y']
figsize = (10,5)


results = {}
spIndex = np.zeros([len(analysis.class_labels), analysis.parameters["HyperParameters"]["n_folds"], len(neurons_mat)])

if not os.path.exists(analysis_file):
    for inovelty in range(len(analysis.class_labels)):
        folds = range(len(analysis.CVO[inovelty]))
        for ifold in folds:    
            class_eff_mat = np.zeros([analysis.parameters["HyperParameters"]["n_folds"],len(np.unique(all_trgt))])
            known_sp_mat = np.zeros([analysis.parameters["HyperParameters"]["n_folds"]])

            buff = np.zeros([len(np.unique(all_trgt))-1])
            class_eff = np.zeros([len(np.unique(all_trgt))], dtype=object)
            known_sp = np.zeros([len(np.unique(all_trgt))], dtype=object)

            def getSP(ineuron):
                train_id, test_id = analysis.CVO[inovelty][ifold]

                # normalize known classes
                if analysis.parameters["HyperParameters"]["norm"] == "mapstd":
                    scaler = preprocessing.StandardScaler().fit(analysis.trn_data[inovelty][train_id,:])
                elif analysis.parameters["HyperParameters"]["norm"] == "mapstd_rob":
                    scaler = preprocessing.RobustScaler().fit(analysis.trn_data[inovelty][train_id,:])
                elif analysis.parameters["HyperParameters"]["norm"] == "mapminmax":
                    scaler = preprocessing.MinMaxScaler().fit(analysis.trn_data[inovelty][train_id,:])

                known_data = scaler.transform(analysis.trn_data[inovelty][test_id,:])
                known_trgt = analysis.trn_trgt[inovelty][test_id]
                
                classifier = SAE[inovelty].load_classifier(data  = analysis.trn_data[inovelty],
                                                           trgt  = analysis.trn_trgt[inovelty], 
                                                           hidden_neurons = hidden_neurons[:layer-1]+[ineuron],
                                                           layer = layer,
                                                           ifold = ifold)

                output = classifier.predict(known_data)
                
                num_known_classes = trn_trgt_sparse[inovelty].shape[1]
                thr_value = 0.2
                for iclass, class_id in enumerate(np.unique(all_trgt)):
                    if iclass == inovelty:
                        continue
                    output_of_class_events = output[known_trgt==iclass-(iclass>inovelty),:]
                    correct_class_output = np.argmax(output_of_class_events,axis=1)==iclass-(iclass>inovelty)
                    output_above_thr = output_of_class_events[correct_class_output,iclass-(iclass>inovelty)]>thr_value
                    class_eff = float(np.sum(output_above_thr))/float(output_of_class_events.shape[0])
                    buff[iclass-(iclass>inovelty)] = class_eff

                sp_index = (np.sqrt(np.mean(buff,axis=0)*np.power(np.prod(buff),1./float(len(buff)))))

                
                return ineuron, sp_index

            # Start Parallel processing
            p = multiprocessing.Pool(processes=num_processes)

            if verbose:
                print '[*] Calculating SP Index ...'
            try:
                results = p.map(getSP, neurons_mat)
                p.close()
                p.join()
                for ineuron_index in range(len(neurons_mat)):
                    spIndex[inovelty, ifold, neurons_mat.index(results[ineuron_index][0])] = results[ineuron_index][1]
            except Exception as e: 
                p.close()
                p.join()          
                print("Erro: {}".format(str(e)))
                break
    joblib.dump([neurons_mat,spIndex],analysis_file,compress=9)
else:
    [neurons_mat, spIndex] = joblib.load(analysis_file)

    
for inovelty in range(len(analysis.class_labels)):
    # Plot results    
    fig = plt.subplots(figsize=figsize)
    ax = plt.subplot(1,1,1)
    
    mean_sp = np.mean(spIndex[inovelty,:], axis=0)
    error_sp = np.std(spIndex[inovelty,:,:], axis=0)
    
    ax.plot(neurons_mat, mean_sp, color='b', alpha=0.7, linewidth=2.5, label='SP Index Test Data')
    
    ax.fill_between(neurons_mat, mean_sp+error_sp, mean_sp-error_sp, facecolor='blue', alpha=0.3)
    
    ax.set_title('SP Index x Neurons (Class {} as novelty)'.format(analysis.class_labels[inovelty]),
                                  fontsize=14, fontweight='bold')
    ax.set_ylabel('SP Index', fontsize=22)
    ax.set_xlabel('Neurons', fontsize=22)
    ax.grid()
    ax.legend()
    plt.tight_layout()
plt.show()

In [ ]:
from sklearn import preprocessing

inovelty = 0
iclass = 1
ifold = 2

train_id, test_id = analysis.CVO[inovelty][ifold]

# normalize known classes
if analysis.parameters["HyperParameters"]["norm"] == "mapstd":
    scaler = preprocessing.StandardScaler().fit(analysis.trn_data[inovelty][train_id,:])
elif analysis.parameters["HyperParameters"]["norm"] == "mapstd_rob":
    scaler = preprocessing.RobustScaler().fit(analysis.trn_data[inovelty][train_id,:])
elif analysis.parameters["HyperParameters"]["norm"] == "mapminmax":
    scaler = preprocessing.MinMaxScaler().fit(analysis.trn_data[inovelty][train_id,:])

known_data = scaler.transform(analysis.trn_data[inovelty][test_id,:])
known_trgt = analysis.trn_trgt[inovelty][test_id]

print np.unique(known_trgt)

In [ ]:
inovelty = 0
iclass = 2
ifold = 2

train_id, test_id = analysis.CVO[inovelty][ifold]

# normalize known classes
if analysis.parameters["HyperParameters"]["norm"] == "mapstd":
    scaler = preprocessing.StandardScaler().fit(analysis.trn_data[inovelty][train_id,:])
elif analysis.parameters["HyperParameters"]["norm"] == "mapstd_rob":
    scaler = preprocessing.RobustScaler().fit(analysis.trn_data[inovelty][train_id,:])
elif analysis.parameters["HyperParameters"]["norm"] == "mapminmax":
    scaler = preprocessing.MinMaxScaler().fit(analysis.trn_data[inovelty][train_id,:])

known_data = scaler.transform(analysis.trn_data[inovelty][test_id,:])
known_trgt = analysis.trn_trgt[inovelty][test_id]

print known_trgt[known_trgt==iclass]

## Receiver Operating Characteristic (ROC) Curve for SP/Trigger with Novelty Detection

### Figures of Merit for a threshold sweep at output layer

In [ ]:
# Thresolds variation x Figures of Merit
%matplotlib inline 

from sklearn import metrics
from sklearn import preprocessing
from sklearn.externals import joblib
from Functions.StatisticalAnalysis import KLDiv, EstPDF
from Functions import FunctionsDataVisualization
import matplotlib.pyplot as plt

# Choose layer 
layer = 1

# Choose neurons topology
hidden_neurons = [200]

analysis_name = 'figures_of_merit_%i_layer'%(layer)
analysis_file = os.path.join(analysis.getBaseResultsPath(), "AnalysisFiles", analysis_name + ".jbl")    

verbose = True

# Plot parameters
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['legend.numpoints'] = 1
plt.rcParams['legend.handlelength'] = 3
plt.rcParams['legend.borderpad'] = 0.3
plt.rcParams['legend.fontsize'] = 14
m_colors = ['b', 'r', 'g', 'y']
figsize = (20,15)


if not os.path.exists(analysis_file):
    # Set the threshold to be analyzed
    thr_mat = np.round(np.arange(0.0,1.1,0.1),3)
    thr_mat[thr_mat>-0.1] = abs(thr_mat[thr_mat>-0.1])
    
    n_folds = analysis.n_folds
    
    class_eff_mat = np.zeros([n_folds,len(np.unique(all_trgt)),len(np.unique(all_trgt)),len(thr_mat)])
    novelty_eff_mat = np.zeros([n_folds,len(np.unique(all_trgt)),len(thr_mat)])
    known_acc_mat = np.zeros([n_folds,len(np.unique(all_trgt)),len(thr_mat)])
    known_sp_mat = np.zeros([n_folds,len(np.unique(all_trgt)),len(thr_mat)])
    known_trig_mat = np.zeros([n_folds,len(np.unique(all_trgt)),len(thr_mat)])
    
    for inovelty, novelty_class in enumerate(np.unique(analysis.all_trgt)):
        for ifold in range(len(analysis.CVO[inovelty])):
            train_id, test_id = analysis.CVO[inovelty][ifold]
            
            print 'Novelty class: %01.0f - Topology: %s - fold %i'%(novelty_class,
                                                                    models[inovelty].get_neurons_str(data=trn_data[inovelty], hidden_neurons=hidden_neurons)+'x'+str(trn_trgt_sparse[inovelty].shape[1]),
                                                                    ifold)
            classifier = SAE[inovelty].load_classifier(data  = analysis.trn_data[inovelty],
                                                       trgt  = analysis.trn_trgt[inovelty], 
                                                       hidden_neurons = hidden_neurons[:layer-1]+[ineuron],
                                                       layer = layer,
                                                       ifold = ifold)
            
            # normalize known classes
            if analysis.parameters["HyperParameters"]["norm"] == "mapstd":
                scaler = preprocessing.StandardScaler().fit(all_data[all_trgt!=inovelty][train_id,:])
            elif analysis.parameters["HyperParameters"]["norm"] == "mapstd_rob":
                scaler = preprocessing.RobustScaler().fit(all_data[all_trgt!=inovelty][train_id,:])
            elif analysis.parameters["HyperParameters"]["norm"] == "mapminmax":
                scaler = preprocessing.MinMaxScaler().fit(all_data[all_trgt!=inovelty][train_id,:])

            known_data = scaler.transform(analysis.trn_data[inovelty][test_id,:])
            known_trgt = analysis.trn_trgt[inovelty][test_id]
            
            novelty_data = scaler.transform(all_data[all_trgt==inovelty])
            
            output = classifier.predict(known_data)
            novelty_output = classifier.predict(novelty_data)
            
            for ithr,thr_value in enumerate(thr_mat): 
                buff = np.zeros([len(np.unique(all_trgt))-1])
                for iclass, class_id in enumerate(np.unique(all_trgt)):
                    if iclass == inovelty:
                        continue
                    output_of_class_events = output[known_trgt==iclass-(iclass>inovelty),:]
                    correct_class_output = np.argmax(output_of_class_events,axis=1)==iclass-(iclass>inovelty)
                    output_above_thr = output_of_class_events[correct_class_output,iclass-(iclass>inovelty)]>thr_value
                    class_eff_mat[ifold, inovelty, iclass, ithr] = float(sum(output_above_thr))/float(len(output_of_class_events))
                    buff[iclass-(iclass>inovelty)] = class_eff_mat[ifold, inovelty, iclass, ithr]
                novelty_eff_mat[ifold, inovelty, ithr] = float(sum(1-(novelty_output>thr_value).any(axis=1)))/float(len(novelty_output))
                known_acc_mat[ifold, inovelty, ithr] = np.mean(buff,axis=0)
                known_sp_mat[ifold, inovelty, ithr]= (np.sqrt(np.mean(buff,axis=0)
                                                              *np.power(np.prod(buff),1./float(len(buff)))))
                known_trig_mat[ifold, inovelty, ithr]=float(sum(np.max(output,axis=1)>thr_value))/float(len(output))
    joblib.dump([class_eff_mat, novelty_eff_mat, known_acc_mat, known_sp_mat, known_trig_mat, thr_mat],
                analysis_file,compress=9)
else:
    print 'file exists'
    [class_eff_mat, novelty_eff_mat, known_acc_mat, known_sp_mat, known_trig_mat, thr_mat] = joblib.load(analysis_file) 

# plot analysis
import matplotlib.pyplot as plt
%matplotlib inline  

fig = plt.subplots(figsize=figsize)

for inovelty, novelty_class in enumerate(np.unique(all_trgt)):
    ax = plt.subplot(2,2,inovelty+1)
    for iclass, m_class in enumerate(np.unique(all_trgt)):
        if novelty_class == m_class:
            #a = 0
            ax.errorbar(thr_mat,np.mean(novelty_eff_mat[:,int(novelty_class),:],axis=0),
                        np.std(novelty_eff_mat[:,int(novelty_class),:],axis=0),fmt='o-',
                        color='k',alpha=0.7,linewidth=2.5,
                        label='Novel Det.')
            ax.errorbar(thr_mat,np.mean(known_acc_mat[:,int(novelty_class),:],axis=0),
                        np.std(known_acc_mat[:,int(novelty_class),:],axis=0),fmt='o--',
                        color='k',alpha=0.7,linewidth=2.5,
                        label='Known Acc.')
            ax.errorbar(thr_mat,np.mean(known_sp_mat[:,int(novelty_class),:],axis=0),
                        np.std(known_sp_mat[:,int(novelty_class),:],axis=0),fmt='o:',
                        color='k',alpha=0.7,linewidth=2.5,
                        label='Known SP')
#             ax.errorbar(thr_mat,np.mean(known_trig_mat[:,int(novelty_class),:],axis=0),
#                         np.std(known_trig_mat[:,int(novelty_class),:],axis=0),fmt='o-.',
#                         color='k',alpha=0.7,linewidth=2.5,
#                         label='Known Trig.')
        else:
            ax.errorbar(thr_mat,np.mean(class_eff_mat[:,int(novelty_class),int(m_class),:],axis=0),
                        np.std(class_eff_mat[:,int(novelty_class),int(m_class),:],axis=0),fmt='o-',
                        color=m_colors[int(m_class)],alpha=0.7,linewidth=2.5,
                       label='C%i Eff.'%(int(m_class)+1))
    ax.set_xticks(thr_mat)
    ax.set_xticklabels(thr_mat,rotation=45, fontsize=18)
    ax.set_title('Eff per Known Class',fontsize=18,weight='bold')
    ax.set_xlim([np.min(thr_mat), np.max(thr_mat)])
    
    ax.set_ylim([0.0, 1.3])
    y_ticks = np.arange(0.0,1.3,0.1)
    ax.set_yticks(y_ticks)
    y_tick_labels = 100*y_ticks[y_ticks<=1.0]
    y_tick_labels = y_tick_labels.astype(int)
    ax.set_yticklabels(y_tick_labels,fontsize=18)
    
    ax.grid()
    
    if inovelty > 1:
        ax.set_xlabel('Threshold',fontsize=18,weight='bold')
    if inovelty == 0 or inovelty == 2:
        ax.set_ylabel('Figures-of-Merit (%)',fontsize=18,weight='bold')
        
    handles, labels = ax.get_legend_handles_labels()
    # sort both labels and handles by labels
    labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
    ax.legend(handles, labels, ncol=3, loc='upper center')
   
    plt.tight_layout()